In [1]:
#Importar librerías para tratamiento de datos
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
import re
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

from selenium.common.exceptions import NoSuchElementException



In [7]:
# Carga archivo donde tenemos todas las peliculas
df_pelis = pd.read_csv("peliculas_api.csv")

# los ID son string
df_pelis["ID de la Película"] = df_pelis["ID de la Película"].astype(str)

# abre google
driver = webdriver.Chrome()

# maximiza
driver.maximize_window()

# Itera sobre los ID
for id_imdb in df_pelis["ID de la Película"][:3]:
    try:
        # Cargamos la página de IMDB
        url = f"https://www.imdb.com/title/{id_imdb}/"
        driver.get(url)

        sleep(3)

        # Aceptamos las cookies
        try:
            driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
        except NoSuchElementException:
            pass   

        # Hacemos clic en el enlace "Reparto y equipo"
        driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button').click()
        sleep(2)

         #Vamos a reparto y equipo
        driver.find_element("xpath", "/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a").click()
        sleep(2)

        # Inicializamos una lista para almacenar las tuplas de actores
        lista_actores = []

        # Iteramos sobre los 10 primeros actores
        for i in range(2, 12):
            try:
                #Sacamos el nombre de la peli
                try: 
                    nombre_peli = driver.find_element("xpath", '//*[@id="main"]/div[1]/div[1]/div/div[1]/h3/a').text
                except NoSuchElementException:
                    nombre_peli = "N/A"

                # Hacemos clic en el enlace de cada actor o actriz
                driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
                sleep(3)

                # Sacamos el nombre de actr
                nombre_act = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div > h1 > span").text

                #Extraemos las pelis por las que se le conoce
                conocido_por1 = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(1) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a").text
                conocido_por2 = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-a6d4b6c0-0.bBRhdF > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child(2) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top > a").text
                conocido_por = [conocido_por1, conocido_por2] if conocido_por2 != "N/A" else [conocido_por1]

                # Extraemos el año de nacimiento del actor o actriz 
                try:        
                  #año_nacimiento_texto = driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-4.eqrmzR > div.sc-92625f35-6.cApiRN > div.sc-92625f35-11.qOBXf > section > aside > div > span:nth-child(2)").text
                  año_nacimiento_texto = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div/span[2]').text
                  # Extraemos solo el año
                  año_nacimiento = año_nacimiento_texto.split()[-1]           
                except NoSuchElementException:
                    try:
                     año_nacimiento= driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-4.brwmyD > div > div.sc-92625f35-11.qOBXf > section > aside > div > span:nth-child(2)").text
                     # Extraemos solo el año
                     año_nacimiento = año_nacimiento_texto.split()[-1]   
                    except NoSuchElementException:
                       año_nacimiento= "N/A"
                       try:
                           año_nacimiento= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div/div[2]/section/aside/div/span[2]').text
                       except NoSuchElementException:
                           año_nacimiento= "N/A"
                           
                # Ahora la profesión 
                profesiones = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text.split("\n")
                sleep(2)
                # Los premios del actor o actriz, si los tiene
                try:
                    #ESTE HA CAMBIADO EL 12DEMAYO -- premios = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/div/ul/li/span').text                 
                    premios = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul/li/span').text
                except NoSuchElementException:
                    premios = "N/A"

                # Creamos la tupla de actor y agregamos
                actor_info = (nombre_peli, nombre_act, año_nacimiento, conocido_por, profesiones, premios)
                lista_actores.append(actor_info)-

            except NoSuchElementException:
                pass  # Si no se encontró el enlace del actor, continuamos con el siguiente

            finally:
                # Regresa a la página anterior para seleccionar al siguiente actor
                driver.back()
                sleep(2)

        # Imprimimos los datos de los actores en forma de lista de tuplas
        print("Datos de los actores:")
        print(lista_actores)

    except Exception as e:
        print(f"Error al procesar la película con ID de IMDb {id_imdb}: {e}")

# Cerramos el WebDriver al finalizar
driver.quit()


Datos de los actores:
[('Pál Adrienn', 'Éva Gábor', 'N/A', ['Europa, East', 'Pál Adrienn'], ['Reparto'], 'N/A'), ('Pál Adrienn', 'István Znamenák', '1966', ['Az állampolgár', 'Taxidermia'], ['Reparto', 'Dirección', 'Dirección de fotografía y cámara'], 'N/A'), ('Pál Adrienn', 'Ákos Horváth', '1968', ['Kontroll', 'Haunting Trophies'], ['Reparto', 'Guion'], 'N/A'), ('Pál Adrienn', 'Lia Pokorny', '1971', ['Ízig-vérig', 'Well'], ['Reparto'], 'N/A'), ('Pál Adrienn', 'Izabella Hegyi', 'N/A', ['Friss levegö', 'Pál Adrienn'], ['Reparto'], 'N/A'), ('Pál Adrienn', 'Szabolcs Thuróczy', '1971', ['Matones sobre ruedas', 'Sandwich'], ['Reparto', 'Guion'], '3 premios'), ('Pál Adrienn', 'Ági Margitai', '1937', ['Ilusiones de un mentiroso', 'Ripacsok'], ['Reparto'], 'N/A'), ('Pál Adrienn', 'Eszter Márton', '1937', ['Nunca moriremos (Sose halunk meg)', 'Jóban Rosszban'], ['Guion', 'Reparto'], 'N/A'), ('Pál Adrienn', 'Lóránt Matta', '1937', ['Pál Adrienn', 'Mi kérünk elnézést!'], ['Reparto'], 'N/A'), ('Pá

In [9]:
df_actores = pd.DataFrame(lista_actores, columns=["Película","Nombre Actor/Actriz", "Año de nacimiento", "Películas conocidas", "Profesiones", "Premios"])

# Imprimir el DataFrame
print("DataFrame de actores:")
df_actores

DataFrame de actores:


,Película,Nombre Actor/Actriz,Año de nacimiento,Películas conocidas,Profesiones,Premios
0,Los hijos del dragón verde,Zoltán Rátóti,1960,"[Európa expressz, Életképek]",[Reparto],N/A
1,Los hijos del dragón verde,Yu Debin,1960,"[Love, Death & Robots, A tökéletes gyilkos]","[Reparto, Efectos visuales]",N/A
2,Los hijos del dragón verde,Eszter Bánfalvi,1960,"[Strike Back, Hamvadó cigarettavég]",[Reparto],Eszter Bánfalvy
3,Los hijos del dragón verde,Ferenc Lengyel,1960,"[Strike Back, Curtiz]","[Reparto, Guion]",N/A
4,Los hijos del dragón verde,Attila Egyed,1960,"[El niño con el pijama de rayas, Tall Tales]","[Reparto, Arte]",N/A
5,So Much for Justice!,Simon Szabó,1979,"[Moszkva tér, Zuhanórepülés]","[Reparto, Guion, Dirección]",5 premios y 1 nominación
6,So Much for Justice!,Lajos Balázsovits,1946,"[Milarepa, Vicios privados, virtudes públicas]","[Reparto, Dirección, Banda sonora]",N/A


In [12]:
df_actores.tail(15)

,Película,Nombre Actor/Actriz,Año de nacimiento,Películas conocidas,Profesiones,Premios
0,Los hijos del dragón verde,Zoltán Rátóti,1960,"[Európa expressz, Életképek]",[Reparto],N/A
1,Los hijos del dragón verde,Yu Debin,1960,"[Love, Death & Robots, A tökéletes gyilkos]","[Reparto, Efectos visuales]",N/A
2,Los hijos del dragón verde,Eszter Bánfalvi,1960,"[Strike Back, Hamvadó cigarettavég]",[Reparto],Eszter Bánfalvy
3,Los hijos del dragón verde,Ferenc Lengyel,1960,"[Strike Back, Curtiz]","[Reparto, Guion]",N/A
4,Los hijos del dragón verde,Attila Egyed,1960,"[El niño con el pijama de rayas, Tall Tales]","[Reparto, Arte]",N/A
5,So Much for Justice!,Simon Szabó,1979,"[Moszkva tér, Zuhanórepülés]","[Reparto, Guion, Dirección]",5 premios y 1 nominación
6,So Much for Justice!,Lajos Balázsovits,1946,"[Milarepa, Vicios privados, virtudes públicas]","[Reparto, Dirección, Banda sonora]",N/A


In [ ]:
#COMPROBANDO LOS DATOS EXTRAÍDOS DE PAL ADRIEN QUE NO APARECEN EN EL DF
#EN EFECTO SON 10 ENTRADAS, NO ENTIENDO EL PROBLEMA
#('Pál Adrienn', 'Éva Gábor', 'N/A', ['Europa, East', 'Pál Adrienn'], ['Reparto'], 'N/A'), 
#('Pál Adrienn', 'István Znamenák', '1966', ['Az állampolgár', 'Taxidermia'], ['Reparto', 'Dirección', 'Dirección de fotografía y cámara'], 'N/A'), 
#('Pál Adrienn', 'Ákos Horváth', '1968', ['Kontroll', 'Haunting Trophies'], ['Reparto', 'Guion'], 'N/A'), 
#('Pál Adrienn', 'Lia Pokorny', '1971', ['Ízig-vérig', 'Well'], ['Reparto'], 'N/A'), 
#('Pál Adrienn', 'Izabella Hegyi', 'N/A', ['Friss levegö', 'Pál Adrienn'], ['Reparto'], 'N/A'), 
#('Pál Adrienn', 'Szabolcs Thuróczy', '1971', ['Matones sobre ruedas', 'Sandwich'], ['Reparto', 'Guion'], '3 premios'), 
#('Pál Adrienn', 'Ági Margitai', '1937', ['Ilusiones de un mentiroso', 'Ripacsok'], ['Reparto'], 'N/A'), 
#('Pál Adrienn', 'Eszter Márton', '1937', ['Nunca moriremos (Sose halunk meg)', 'Jóban Rosszban'], ['Guion', 'Reparto'], 'N/A'), 
#('Pál Adrienn', 'Lóránt Matta', '1937', ['Pál Adrienn', 'Mi kérünk elnézést!'], ['Reparto'], 'N/A'), 
#('Pál Adrienn', 'Róbert Kardos', '1937', ['Last Chance', 'Papsajt'], ['Reparto', 'Dirección', 'Guion'], 'N/A')

In [ ]:
#NO TOCAR!!!

#NO EJECUTAR ESTA CELDA, SERÁ PARA GUARDAR LA INFO POSTERIORMENTE

#def signal_handler(signal, frame, driver, datos_peliculas):
#    print("Proceso interrumpido. Guardando datos...")
#    df_resultados_temporales = pd.DataFrame(datos_peliculas)
#    df_resultados_temporales.to_csv('fase3Datos.csv', index=False)
#    print("Datos guardados en 'fase3Datos.csv'.")
    guardar_datos(datos_peliculas)  #  definir esta función
    driver.close()
    sys.exit(0)
    
def guardar_datos(datos_peliculas):
   # Define el  archivo donde vamos a guardar los datos.
    ruta_archivo = 'fase-3-datos-reparto-imdb.csv'
    
    # Convierte el diccionario a DataFrame
    df_nuevos_datos = pd.DataFrame(datos_peliculas)
    
    if os.path.exists(ruta_archivo):
        # Si el archivo ya existe, carga los datos existentes y los junta.
        df_existentes = pd.read_csv(ruta_archivo)
        df_resultante = pd.concat([df_existentes, df_nuevos_datos], ignore_index=True)
    else:
        # Si el archivo no existe, utiliza los nuevos datos directamente
        df_resultante = df_nuevos_datos
    
    # Guardamos el DataFrame resultante en el archivo, sobrescribiendo el archivo existente
    df_resultante.to_csv(ruta_archivo, index=False)
    
    print(f"Datos guardados o actualizados exitosamente en {ruta_archivo}.")